# Modeling perturbations

The wetlab schema is designed to model complex <a href="../wetlab.experiment.html">wetlab.Experiment</a> of any <a href="../wetlab.experimenttype.html">wetlab.ExperimentType</a> that include perturbations in the experimental design.

Perturbations, here also termed 'treatment', are intentional disruptions in biological systems, such as <a href="../wetlab.genetictreatment.html">wetlab.GeneticTreatment</a>, <a href="../wetlab.compoundtreatment.html">wetlab.CompoundTreatment</a>, or <a href="../wetlab.environmentaltreatment.html">wetlab.EnvironmentalTreatment</a>, to study their effects on molecular and cellular processes.
Such treatments sometimes also have known <a href="../wetlab.treatmenttarget.html">wetlab.TreatmentTarget</a> which can be one or serveral {class}`~docs:bionty.Gene`, {class}`~docs:bionty.Pathway`, or {class}`~docs:bionty.Protein`.
While single treatments are common, it is also possible to combine several treatments into a <a href="../wetlab.combinationtreatment.html">wetlab.CombinationTreatment</a>.

In [ ]:
#! pip install 'lamindb[bionty]' wetlab
!lamin init --storage modeling-perturbations --schema bionty,wetlab

In [ ]:
import bionty as bt
import wetlab as wl

Gefitinib is primarily a pharmaceutical treatment, specifically a targeted therapy used in cancer treatment, particularly for non-small cell lung cancer (NSCLC).
Let's assume that we want to better understand the effectiveness of gefitinib and EGFR/KRAS knockdown combination treatment for people that have a smoking history.
In our early experimental setup, we therefore subject mice to smoke.

In [ ]:
bt.settings.organism = "mouse"

## Genetic treatments

We create two <a href="../wetlab.genetictreatment.html">wetlab.GeneticTreatment</a> records associated with the corresponding <a href="../wetlab.treatmenttarget.html">wetlab.TreatmentTarget</a>.

In [ ]:
EGFR_kd = wl.GeneticTreatment(
    system="CRISPR-Cas9",
    name="EGFR knockdown",
    sequence="AGCTGACCGTGA",
    on_target_score=85,
    off_target_score=15,
).save()

In [ ]:
EGFR_gene = bt.Gene.from_source(symbol="EGFR").save()
EGFR_kd_target = wl.TreatmentTarget(name="EGFR").save()
EGFR_kd_target.genes.add(EGFR_gene)

In [ ]:
EGFR_kd.targets.add(EGFR_kd_target)
EGFR_kd_target

In [ ]:
KRAS_kd = wl.GeneticTreatment(
    system="CRISPR-Cas9",
    name="KRAS",
    sequence="TTGGTGGTGAACT",
    on_target_score=100,
    off_target_score=20,
).save()

In [ ]:
KRAS_gene = bt.Gene.from_source(symbol="KRAS").save()
KRAS_kd_target = wl.TreatmentTarget(name="KRAS").save()
KRAS_kd_target.genes.add(KRAS_gene)

In [ ]:
KRAS_kd.targets.add(KRAS_kd_target)
KRAS_kd_target

## Compound treatments

Next, we create a <a href="../wetlab.compoundtreatment.html">wetlab.CompoundTreatment</a> records including associated <a href="../wetlab.treatmenttarget.html">wetlab.TreatmentTarget</a>.

In [ ]:
gefitinib = compound_treatment = wl.CompoundTreatment(
    name="gefitinib",
).save()

Gefitinib is a tyrosine kinase inhibitor (TKI) that specifically targets the epidermal growth factor receptor (EGFR) pathway and the EGFR protein.
Therefore, we can also define the <a href="../wetlab.treatmenttarget.html"> and associate it with a :class:`bionty.Pathway` and :class:`bionty.Protein`.

In [ ]:
egfr_pathway = bt.Pathway.from_source(
    name="epidermal growth factor receptor activity"
).save()
egfr_protein = bt.Protein.from_source(uniprotkb_id="Q5SVE7").save()
egfr_targets = wl.TreatmentTarget(name="EGFR").save()
egfr_targets.pathways.add(egfr_pathway)
egfr_targets.proteins.add(egfr_protein)
egfr_targets

In [ ]:
gefitinib.targets.add(egfr_targets)
gefitinib

## Environmental Treatment

In [ ]:
smoking = wl.EnvironmentalTreatment(
    name="smoking status measurement", ontology_id="EFO:0006527"
).save()

## Combination Treatment

Now we can combine all individual treatments into a single <a href="../wetlab.combinationtreatment.html">wetlab.CombinationTreatment</a>.

In [ ]:
combination_treatment = wl.CombinationTreatment(
    name="gefitinib and EGFR/KRAS knockdown combination treatment subject to smoking"
).save()
combination_treatment.genetics.set([EGFR_kd, KRAS_kd])
combination_treatment.compounds.add(gefitinib)
combination_treatment.environmentals.add(smoking)

In [ ]:
combination_treatment